In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader , SubsetRandomSampler
import torch.nn as nn
from collections import defaultdict
import torchvision
import albumentations as albu
from albumentations.pytorch.transforms import ToTensor

import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from torch.utils.data.sampler import Sampler
import sys
from PIL import Image

from sklearn.model_selection import KFold

#import tqdm
try:
    get_ipython().__class_._name__
    from tqdm.notebook import tqdm
except:
    from tqdm import tqdm



In [ ]:
IMG_SIZE         = 510
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
POSTIVE_PERC     = 0.8
DEVICE           = 'cuda'

In [ ]:
class Dataset():
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.dataset = pd.read_csv(self.root)
        
    def __getitem__(self, idx):
                                       # segmentation to results
        image =Image.open(os.path.join('PATH_TO_MASKS', self.dataset['filename'][idx]))
        image =  np.expand_dims(cv2.resize(np.array(image), (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)[(IMG_SIZE//2):,:,0],axis=2)
        label = self.dataset['evaluation'][idx]

       
        if self.transform:
            sample = {"image": image, "label": label}
            sample = self.transform(**sample)
            image = sample['image']#.reshape((3,IMG_SIZE,IMG_SIZE)), 
            label = sample['label']

        

        return {
            'image': image, 
            'label' : label
        }
        
    def __len__(self):
        return len(self.dataset)

In [ ]:
# transforms
TFMS = albu.Compose([
    ToTensor(),
])

# dataset
root = 'PATH TO CSV FILE'
dataset = Dataset(root = root,transform=TFMS) 
loader = DataLoader(dataset, 1, shuffle=False, num_workers=4)



In [ ]:
# check sanity of dataset 
def matplotlib_imshow(img, one_channel=False):
    fig,ax = plt.subplots(figsize=(10,6))
    ax.imshow(img.permute(1,2,0).numpy())
    
def visualize(**images):
    """PLot images in one row."""
    plt.figure(figsize=(16, 8))
    image = np.squeeze(images['image'])

    plt.imshow(image, vmin=0, vmax=1)
     
    plt.show()

In [ ]:

dataset[10]['image'].shape

In [ ]:
# plot one with mask 
visualize(**dataset[10])

In [ ]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(53760, 4096)# for newest
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 1)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.maxpool(x)
        print(x.shape)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)
        return x

In [ ]:

model = VGG16()
model.to('cuda')
model.load_state_dict(torch.load('PATH TO MODEL'))

In [ ]:
true_values = []
predicted_values = []
for i, data in enumerate(loader, 0):
                # get the inputs; data is a list of [inputs, labels]
                input, labels = data['image'].to('cuda'), data['label'].to('cuda')
                if i%100 == 0 :
                  print(i)
                
                output = model(input)
                #loss = torch.sqrt(criterion(item['label'], out))
                output, labels = output.cpu().type(torch.float64).detach().numpy(), labels.cpu().type(torch.float64).detach().numpy() 
                predicted_values.append(output)
                true_values.append(labels)

true_values = [i.squeeze()for i in true_values]                
predicted_values = [i.squeeze()for i in predicted_values]
d = abs(np.array(true_values)-np.array(predicted_values))

In [ ]:
import scipy
import sklearn

plt.hist(abs(d)*20)
acc = sum(d<0.5)/len(d)
from scipy.stats.stats import pearsonr 
from sklearn.metrics import r2_score

err = np.mean(d*20)

print("correlation : "+str(pearsonr(true_values,predicted_values)[0])[:6])
print("Accuracy : "+str(acc)[:6]+"%")
print("Average error  : "+str(err)[:6]+"%")
print("R2   : "+str(r2_score(true_values, predicted_values))[:6])